In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import pandas as pd
import requests
import io

#To import the genetic algorithm method
from genalg import GA_fs

# Importing Dataset

In [2]:
# Downloading the csv files from the GitHub account

url_metabolic = "https://raw.githubusercontent.com/himasai97/ML_Approaches/master/Data/flux.csv"
url_phen = "https://raw.githubusercontent.com/himasai97/ML_Approaches/master/Data/Phenotype_Measurements.csv"


download_metabolic = requests.get(url_metabolic).content
download_phen = requests.get(url_phen).content


# Reading the downloaded content and turning it into a pandas dataframes

metabolic = pd.read_csv(io.StringIO(download_metabolic.decode('utf-8'))) #Input metabolic data of fluxes and metabolites
y = pd.read_csv(io.StringIO(download_phen.decode('utf-8'))) #Output Phenotype values

#Extracting data relevant to phenotypes under study
#because the study is to understand the effect of transgenic modifications on the traits of the plant,
#height, volume and diameter are more relevant to the study than their growth rates. Hence the three rates are being dropped.
y = y.drop(columns=['Row_1', 'HeightRate','DiameterRate', 'VolumeRate']) #removing the label, and the three irrelevant phenotypes

#Extracting data relevant to input flux

l =list(range(1,16))+list(range(17,20))+list(range(22,26))+list(range(27,38))+["12R","14R"];
FluxNames = ["V"+str(num1) for num1 in l]; #35 fluxes
MetNames = ["Y"+str(num2) for num2 in range(2,25)];

flux = metabolic[FluxNames];
X = flux.round(9);

#Getting a list of the Phenotype names
Phen_arr = y.columns;

#Check the size of the data: X-(#samples,#fluxes) and y-(#samples,#phenotypes), where #fluxes=35, #phenotypes=25
print ("X has {} sample points corresponding to the {} fluxes.".format(*X.shape))
print ("y has {} sample points corresponding to the {} phenotypes.".format(*y.shape))


X has 225 sample points corresponding to the 35 fluxes.
y has 225 sample points corresponding to the 25 phenotypes.


# Preparing Dataset

## Handling NaN Values
Missing values in the phenotype measurements are represented as NaNs. For preprocessing, we remove the rows in the flux data that corresponding to the missing values in phenotype measurements. This function is implemented using a user defined routine: **rem_nan**.

In [ ]:
#For Removing NaN
def rem_nan(X,y,Phen):
    y_i = y[Phen]
    #extacting index positions where y_i has NaN values
    idx = y_i.index[y_i.apply(np.isnan)] #to get the row names
    Nan_idx=[(np.where(y.index==idx[i])[0][0]) for i in range(len(idx))] #to get the row numbers for X_i
    X_i = X.drop(Nan_idx,axis =0)
    y_i = y_i.drop(idx,axis =0)
    return X_i, y_i

## Scaling the Data
Most of the machine learning algorithms are designed with the assumption that all independent features present in the data vary on comparable scales. So, when the input features have very different scales, it can degrade the predictive performance of these algorithms. Unscaled data can also lead to longer times or even prevent convergence for many gradient-based estimators. Thus, scaling the data is an important step in data preprocessing for many ML approaches. Among the many scikit-learn methods available for this purpose, we use **MinMaxScaler** as it preserves the shape of the original distribution.

For each value in an input feature, MinMaxScaler subtracts the minimum value in the feature and then divides by the range. The range is the difference between the maximum and minimum values in the feature. The values returned by MinMaxScaler are thus between 0 and 1.

### Implementation
In a typical machine learning workflow, we need to apply the above transformations at least twice. Once when training the model and again on any new data we want to predict on. To automate this workflow, we use Scikit-learn **Pipeline** method, as it sequentially applies the transformer and the estimator on a given dataset. The Pipeline approach is especially useful when applying cross-validation where each set of training data needs to be transformed before being fed into the ML estimator. 


# Hyperparamter Optimization
For a machine learning algorithm, hyperparameters are set before training and supplied to the model. They are different from the model parameters that are learned during training by the machine. Hyperparameters are tuned by the user to achieve optimal model performance. We use **GridSearchCV**, a library function from sklearn’s *model_selection package*. From a given grid of hyperparameter values, defined using the *param-grid* parameter, it exhaustively loops through all combinations to optimize the given model (*estimator*) using cross-validation. The performance of the selected hyper-parameters and the trained model is then measured on a dedicated evaluation set that was not used during the model selection step. The number of folds used in the cross-validation (*cv*) and the metric used for evaluation (*scoring*) are both defined along with the *estimator* and the *param-grid* parameters in the GridSearchCV function.


In [ ]:
# Setting up parameters for the ML approach

# Defining a 10-repeat 5-fold Cross-validation
k_fold = RepeatedKFold(n_splits=5, n_repeats=10)

#Defining model learner for the three ML approaches - learner
l_SVR = SVR(kernel='rbf')
l_XGB = XGBRegressor() 
l_KNN = neighbors.KNeighborsRegressor()

learner = [l_XGB, l_KNN, l_SVR]

# Defining a grid of hyperparameter values for each ML approach - tuned_param
t_SVR = [{'learner__epsilon':[0.05,0.01,0.1,0.001],'learner__gamma': [5,0.1,0.8,50], 'learner__C': [5,80, 1000, 5000]}]

t_XGB = [{"learner__learning_rate"    : [0.05, 0.10, 0.20, 0.30 ] ,
           "learner__max_depth"        : [3,4,5,6,8,12 ],
           "learner__min_child_weight" : [7,5,3,1],
           "learner__gamma"  : [ 0.0,0.1,0.3,0.4  ],
           "learner__colsample_bytree" : [ 0.3,0.4,0.5,0.7 ] }]

t_KNN = [{'learner__n_neighbors': list(range(1, 21)),'learner__weights':['uniform','distance'],'learner__metric':['euclidean','manhattan']}]

tuned_param = [t_XGB, t_KNN, t_SVR]

#Selecting the ML approach to be implemented using its index position in the learner and tuned_param arrays - ml_idx
ml_idx = int(input('Type the integer corresponding to the ML model to be implemented: 0 for XGB, 1 for kNN, 2 for SVR'))

#For Scaling the Data
min_max_scaler = MinMaxScaler()

# Defining the pipeline object with a scaler and a leaner model.
steps = [('scaler', min_max_scaler), ('learner', learner[ml_idx])]
pipeline = Pipeline(steps) 

# Implementation of GridSearch to tune the pipeline method for best score 
# from the given grid of hyper-parameters using cross-validation.
clf = GridSearchCV(estimator=pipeline,
       param_grid= tuned_param[ml_idx], n_jobs=-1, cv=k_fold,scoring ='r2',return_train_score=True)

# Here setting n-jobs as -1 ensures that all available processors are used in the GridSearch implementation 
# and as the name implies, return_train_score is set to True to return the training scores evaluated for the different folds.



# Genetic Algorithm
In version-1 of the ML implementation, we described the ML approaches designed to predict the 25 lignin and wood properties using the 35 monolignol steady-state fluxes as input. Here, we proposed the implementation of genetic algorithm, a feature selection technique, to further improve the performance of these models. In this implementation each chromosome in the population is represented by 35 binary genes for the 35 input fluxes. Here, the binary value of each gene represents whether the corresponding feature is to be included in the model or not.

### Implementation
Genetic Algorithm for Feature Selection: **GA_fs** with inputs:

- *X*: input with all fluxes
- *y*: true values of the predicted phenotype
- *clf*: the ML model to be tuned and used for prediction
- *Phen*: Name of the phenotype to be predicted

and outputs:

- *F_array*: array of selected features 
- *best_clf*: ML model optimized for the selected set of input features
- *train_r2*: R<sup>2</sup> evaluated for the cross-validation folds in the grid search algorithm

Using this method, we obtained a set of input features and hyperparameter values for each ML model to obtain better model efficiency when compared with the implementation in Version 1. To measure efficiency, we evaluated the final R<sup>2</sup> score using the untouched test data.

In [ ]:
#Initialise arrays for the different metrics
R2 = []
train_r2_arr = []


#Implementation
for Phen_idx in range(25):

    Phen = Phen_arr[Phen_idx]

    X_i, y_i = rem_nan(X,y,Phen)
    X_train, X_test, y_train, y_test = train_test_split(X_i, y_i, test_size=0.2, random_state=7)


    (F_array,best_clf,train_r2) = GA_fs(X_train,y_train,clf,Phen)
    
    #Saving the model
    model_file = 'model_'+Phen+'.sav'
    pickle.dump(best_clf, open(model_file, 'wb'))
    
    #Saving Selected features
    F_file = 'features_'+Phen+'.data'
    with open(F_file, 'wb') as filehandle:
        # store the data as binary data stream
        pickle.dump(F_array, filehandle)
    
    #To reload the saved model
    loaded_model = pickle.load(open(model_file, 'rb'))
    
    #To reload the saved features array
    with open(F_file, 'rb') as filehandle:
        # read the data as binary data stream
        selected_f = pickle.load(filehandle)

    #For final evaluation of the trained model, test data is used
    X_test_f = X_test[selected_f]
    y_pred = loaded_model.predict(X_test_f)

    R2[Phen_idx] = r2_score(y_test,y_pred) 
    train_r2_arr[Phen_idx] = train_r2
    
    



# Results

The $R^2$ scores are compared with the results from version 1 for each ML approach and visualized below.

## Figure 1: kNN model
<img src="Images/knn_compare.PNG">
          
## Figure 2: SVR model
<img src="Images/svr_compare.PNG">

## Figure 3: XGB model
<img src="Images/xgb_compare.PNG">


## Analysis

For kNN model, there was an increase in R<sup>2</sup> value by at least 0.1 for 12 lignin and wood properties, including lignin content, SG ratio, S subunits, G subunits, $\beta-O-4$ linkage, $\beta-5$ linkage, endgroups, saccharification efficiency of glucose unpretreated samples, saccharification efficiency of xylose unpretreated samples, saccharification efficiency of xylose pretreated samples, height, and diameter (Fig 1). However, for 5 phenotype traits, including H subunits, phydroxybenzoate, CL ratio, aldehydes, and volume, there was a neglible increase in $R^2$ value, by less than 0.02. 

Using Genetic Algorithm, SVR model showed an increase in $R^2$ score by at least 0.1 for 11 lignin and wood properties, including SG ratio, S subunits, Aldhehydes, $\beta-5$ linkage, endgroups, relative density, modulus of elasticity, saccharification efficiency of glucose pretreated samples, saccharification efficiency of xylose pretreated samples, height, and volume (Fig 2). 

For 4 lignin and wood properties, including H subunits, phydroxybenzoate, xylose, and total sugar, there was very little increase in $R^2$ score, by less than 0.02. Similarly for XGB model, we found that the $R^2$ value increased by at least 0.1 for 13 lignin and wood properties, including SG ratio, S subunits, G subunits, CL ratio, aldehydes, $\beta-O-4$ linkage, $\beta-5$ linkage, $\beta-1$ linkage, height, diameter, volume, relative density, and saccharification efficiency of xylose pretreated samples (Fig 3). However for H subunits and $\beta-\beta$ linkage, there was a negligible increase in $R^2$ values for XGB model, by less than 0.02. 

We can infer that the lack of improvement in $R^2$ values of ML models for certain phenotype traits, even with the reduction in the number of input features, is an indicator of a poor fit by the models in the prediction of these phenotype traits.

# Comparison with the Original Model

Based on the results illustrated in Fig 4, the original random forest method performed better than the ML approaches for only 1 phenotype trait, H subunits. We see that XGB performed better for 8 lignin and wood properties, including SG ratio, phydroxybenzoate, aldehydes, glucose, total sugar, saccharification efficiency of glucose unpretreated samples, saccharification efficiency of glucose pretreated samples, and saccharification efficiency of xylose pretreated samples. For 7 of these lignin and wood properties, except total sugar, SVR also performed better than the original random forest method. kNN, on the other hand, performed better than the original model for only 4 properties, including SG ratio, glucose, saccharification efficiency of xylose unpretreated samples, and saccharification efficiency of glucose pretreated samples. However, all three ML models performed as good as the original random forest model for 7 of the lignin and wood properties (increase in $R^2$ by less than 0.01), including S subunits, G subunits, CL ratio, $\beta-1$ linkage, $\beta-\beta$ linkage, relative density, and modulus of elasticity. 

## Figure 4: With Original Random Forest Method as Baseline Model
<img src="Images/comp_gen.PNG">

Having the original random forest model developed by Matthews as the baseline model, we can conclude that the three machine learning approaches implemented with genetic algorithm produce good predictions for 24 of the 25 phenotype traits.